In [58]:
import pandas as pd
import urllib.parse as urlparse
import json
import re
import requests
import math
import time
from datetime import datetime
bridge_file = open('./Bridge.txt','r')
key_list = bridge_file.readlines()
YT_API_KEY = key_list[0]
PERS_API_KEY = key_list[1]
bridge_file.close()

In [59]:
def startTimeMe(print_flag):
    now = datetime.now()
    dt_string = now.strftime("%H:%M:%S")
    current_time = time.time()
    if print_flag:
        print(f'Start time: {dt_string}')
    #time_list = dt_string.split(':')
    #time_hr,time_min, time_sec = time_list[0],time_list[1],time_list[2]
    return dt_string,current_time

In [67]:
def endTimeMe(input_dataset, label, dt_string_s, start_time):
    dt_string_e,current_time = startTimeMe(False)
    print(f'End time: {dt_string_e}')
    #print(time_hr,time_min, time_sec)
    input_dataset.loc[len(input_dataset.index)] = [label,dt_string_s,dt_string_e,start_time,current_time]
    return dt_string_e,current_time,input_dataset

In [22]:
def getCommentsForVideo(video_id):
    comment_list = []
    count = 0
    comments_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId={video_id}&key={YT_API_KEY}')
    comment_json_data = json.loads(comments_from.text)
    if 'items' in comment_json_data:
        for x in range(0,len(comment_json_data['items'])):
            comment_element = {
                'id': comment_json_data['items'][x]['id'],
                'textOriginal': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['textOriginal'],
                'authorDisplayName': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'likeCount': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['updatedAt']
            }
            comment_list.append(comment_element)
        count = count+len(comment_json_data['items'])
        print(f'{count} processed',end="\r")
        while('nextPageToken' in comment_json_data):
            next_page_token = comment_json_data['nextPageToken']
            comments_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&pageToken={next_page_token}&videoId={video_id}&key={YT_API_KEY}')
            comment_json_data = json.loads(comments_from.text)
            if 'items' in comment_json_data:
                for x in range(0,len(comment_json_data['items'])):
                    comment_element = {
                        'id': comment_json_data['items'][x]['id'],
                        'textOriginal': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['textOriginal'],
                        'authorDisplayName': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                        'likeCount': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['likeCount'],
                        'publishedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['publishedAt'],
                        'updatedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['updatedAt']
                    }
                    comment_list.append(comment_element)
                count = count+len(comment_json_data['items'])
                print(f'{count} processed',end="\r")    
    return count,len(comment_list), comment_list
#getCommentsForVideo('quOHHVVYWlo')

In [23]:
def get_topic_category_from_wiki(wiki_link):
    if 'en.wikipedia.org/wiki/' in wiki_link:
        topic_index = wiki_link.index('wiki/')
        return wiki_link[topic_index+5:]
    else:
        return 'NA'

In [24]:
def convert_time_to_sec(time_str):
    #PT1H15M44S
    total_seconds = 0
    
    if 'DT' in time_str:
        time_str = time_str[1:]
        index = time_str.index('DT')
        day_time = int(time_str[0:index])
        total_seconds = day_time*86400
        time_str = time_str[index+2:]
    else:
        time_str = time_str[2:]
    if 'H' in time_str:
        index = time_str.index('H')
        hour_time = int(time_str[0:index])
        total_seconds = total_seconds+(hour_time*3600)
        time_str = time_str[index+1:]
        #print(hour_time)
        #print(time_str)
    if 'M' in time_str:
        index = time_str.index('M')
        min_time = int(time_str[0:index])
        total_seconds = total_seconds+(min_time*60)
        time_str = time_str[index+1:]
        #print(time_str)
        #print(min_time)
    if 'S' in time_str:
        index = time_str.index('S')
        sec_time = int(time_str[0:index])
        total_seconds = total_seconds+sec_time
        #print(sec_time)
    return total_seconds

In [25]:
def get_video_category_from_id(categoryId):
    category_dict = {'1':'Film & Animation','2':'Autos & Vehicles','10':'Music','15':'Pets & Animals','17':'Sports','19':'Travel & Events',
'20':'Gaming','22':'People & Blogs','23':'Comedy','24':'Entertainment','25':'News & Politics','26':'Howto & Style','27':'Education',
'28':'Science & Technology','29':'Nonprofits & Activism'}
    if categoryId in category_dict:
        return category_dict[categoryId]
    else:
        print(f'{categoryId}: NA')
        return 'NA'

In [26]:
def countUrlType(input_dataset):
    channel_count = 0
    video_count = 0
    other_count = 0
    for idx in input_dataset.index:
        if input_dataset['url_type'][idx] == 'channel':
            channel_count=channel_count+1
        elif input_dataset['url_type'][idx] == 'video':
            video_count=video_count+1
        else:
            other_count=other_count+1
    return channel_count,video_count,other_count

In [27]:
def expanded_url_from(urllist_item):
    if 'expanded_url' in urllist_item:
        index_one = urllist_item.index('expanded_url')+16
        ext_url = ''
        for x in range(index_one,len(urllist_item)):
            if urllist_item[x] == '\'':
                return ext_url
            else:
                ext_url = ext_url+urllist_item[x]
        return urllist_item[index_one:index_two]
    else:
        return urllist_item
#expanded_url_from("[{'url': 'https://t.co/cZdjZpdrGh', 'expanded_url': 'https://youtu.be/ymdrOPAzLqc', 'display_url': 'youtu.be/ymdrOPAzLqc', 'indices': [169, 192]}]")

In [28]:
def checkSuccess(ch_count, vi_count, ot_count, sup_total):
    if (ch_count+vi_count+ot_count) == sup_total:
        return True
    else:
        return False

In [29]:
def getIDfromURL(yt_url):
    link_type = ''
    link_id = ''
    if '/c/' in yt_url or '/channel/' in yt_url:
        if '/c/' in yt_url:
            c_index = yt_url.index('/c/')
            sub_yt_url = yt_url[c_index+3:]
            if '/' in sub_yt_url:
                slash_index = sub_yt_url.index('/')+c_index+3
                link_type = 'channel'
                link_id = yt_url[c_index+3:slash_index]
            elif '?' in sub_yt_url:
                quest_index = sub_yt_url.index('?')+c_index+3
                link_type = 'channel'
                link_id = yt_url[c_index+3:quest_index]                            
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+3:]
            if len(link_id) != 24:
                #make a channel id request
                channelid_response = requests.get(f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=channel&fields=items%2Fsnippet%2FchannelId&q={link_id}&key={YT_API_KEY}')
                channelid_json = json.loads(channelid_response.text)
                if 'items' in channelid_json and len(channelid_json['items']) > 0:
                    if pd.isna(channelid_json['items'][0]['snippet']['channelId']):
                        print(f'true NA {link_id}')
                        link_id='NA'
                    else:
                        link_id = channelid_json['items'][0]['snippet']['channelId']
                else:
                    print(f'{channelid_json}')
                    link_id='NA'
        elif '/channel/' in yt_url:
            c_index = yt_url.index('/channel/')
            sub_yt_url = yt_url[c_index+9:]
            if '/' in sub_yt_url:
                slash_index = sub_yt_url.index('/')+c_index+9
                link_type = 'channel'
                link_id = yt_url[c_index+9:slash_index]
            elif '?' in sub_yt_url:
                quest_index = sub_yt_url.index('?')+c_index+9
                link_type = 'channel'
                link_id = yt_url[c_index+9:quest_index]  
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+9:]
    elif '/watch?v=' in yt_url or '/v=' in yt_url or '/shorts/' in yt_url or '.be/' in yt_url:
        if '/watch?v=' in yt_url:
            w_index = yt_url.index('/watch?v=')
            link_type = 'video'
            link_id = yt_url[w_index+9:w_index+20]
        elif '/v=' in yt_url:
            w_index = yt_url.index('/v=')
            link_type = 'video'
            link_id = yt_url[w_index+3:w_index+14]
        elif '/shorts/' in yt_url:
            w_index = yt_url.index('/shorts/')
            link_type = 'video'
            link_id = yt_url[w_index+8:w_index+19]  
        elif '.be/' in yt_url:
            w_index = yt_url.index('.be/')
            link_type = 'video'
            link_id = yt_url[w_index+4:w_index+15]
    else:
        com_index = yt_url.index('com/')
        if (len(yt_url)-(com_index+4)) == 11:
            link_type = 'video'
            link_id = yt_url[com_index+4:com_index+15]
        else:
            link_type = 'other'
            link_id = 'NA'
    return link_type,link_id

In [30]:
def takeDateQuery():
    month_dict = {'01': 'Jan','02': 'Feb','03': 'Mar','04': 'Apr',
             '05': 'May','06': 'Jun','07': 'Jul','08': 'Aug',
             '09': 'Sep','10': 'Oct','11': 'Nov','12': 'Dec'}
    date_query = input('Input date in MM/DD/YYYY format').split('/')
    month_query = month_dict[date_query[0]]
    day_query = date_query[1]
    year_query = date_query[2]
    print(month_query,day_query,year_query)
    return month_query,day_query,year_query

In [31]:
#Initializing time statistics dataframe

In [70]:
time_df = pd.DataFrame(columns=['Activity','Start Time','End Time','Start (sec)','End (sec)'])

In [71]:
dt_string_s,start_time = startTimeMe(True)
twitter_dataset_onlyurl = pd.read_csv('./datasets/yt_onlyurl.csv')
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Reading \'yt_onlyurl\' dataset',dt_string_s,start_time)
time_df

Start time: 16:49:37
End time: 16:49:44


,Activity,Start Time,End Time,Start (sec),End (sec)
0,Reading 'yt_onlyurl' dataset,16:49:37,16:49:44,1.665532e+09,1.665532e+09


In [57]:
import time
print(round(time.time()))

1665531379


In [49]:
twitter_dataset_onlyurl.columns

Index(['tweetid', 'urls_list', 'url'], dtype='object')

In [47]:
twitter_dataset_onlyurl.size

3908106

In [61]:
twitter_dataset = pd.read_csv('./datasets/yturl_twitter.csv',lineterminator="\n",low_memory=False)

In [62]:
twitter_dataset

,tweetid,userid,screen_name,date,lang,location,description,place_id,place_url,place_type,...,state,country,rt_state,rt_country,qtd_state,qtd_country,norm_country,norm_rt_country,norm_qtd_country,acc_age
0,1495977545967317000,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:35 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,1495977557505753089,2821163219,Billington_Book,Tue Feb 22 04:23:38 +0000 2022,en,"Reus, Tarragonna, Spain","Disabled Vietnam vet, dad, granddad, cancer su...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2691
2,1495977563197476865,294162188,ArsenicMarko,Tue Feb 22 04:23:39 +0000 2022,en,"Aurora, Ontario",NaN,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,3944
3,1495977582155685895,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:44 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
4,1495977589151830017,1378410853314203649,Denise07190935,Tue Feb 22 04:23:45 +0000 2022,nl,NaN,🧠zelfnadenken \n🕊free Humanity \n📚boeken \nexi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302697,1509017366180634626,1331144263657287680,Marek82085289,Wed Mar 30 03:59:10 +0000 2022,en,NaN,PO = KORUPCJA i ZDRAJCY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490
1302698,1509017394299260931,1045050364158853120,life_canbeshort,Wed Mar 30 03:59:17 +0000 2022,en,Canada,The Universe Corrects All Wrongs✌️ pls dont sp...,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,1280
1302699,1509017428831145984,53870131,gusbayu,Wed Mar 30 03:59:25 +0000 2022,in,dipermukaan bumi,Ragadi musuh mepareng \nRihati ya tongwanya ta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4650
1302700,1509017469758832640,777674879701254144,Online_SergioJu,Wed Mar 30 03:59:35 +0000 2022,es,"Santiago, Chile",Saludos ! aquí verás él Mundo a través de mis...,NaN,NaN,NaN,...,NaN,Chile,NaN,NaN,NaN,NaN,Chile,NaN,NaN,2018


In [7]:
twitter_dataset.columns

Index(['tweetid', 'userid', 'screen_name', 'date', 'lang', 'location',
       'description', 'place_id', 'place_url', 'place_type', 'place_name',
       'place_full_name', 'place_country_code', 'place_country',
       'place_bounding_box', 'text', 'extended', 'coord', 'reply_userid',
       'reply_screen', 'reply_statusid', 'tweet_type', 'friends_count',
       'listed_count', 'followers_count', 'favourites_count', 'statuses_count',
       'verified', 'hashtag', 'urls_list', 'profile_pic_url',
       'profile_banner_url', 'display_name', 'date_first_tweet',
       'account_creation_date', 'rt_urls_list', 'mentionid', 'mentionsn',
       'rt_screen', 'rt_userid', 'rt_user_description', 'rt_text',
       'rt_hashtag', 'rt_qtd_count', 'rt_rt_count', 'rt_reply_count',
       'rt_fav_count', 'rt_tweetid', 'rt_location', 'qtd_screen', 'qtd_userid',
       'qtd_user_description', 'qtd_text', 'qtd_hashtag', 'qtd_qtd_count',
       'qtd_rt_count', 'qtd_reply_count', 'qtd_fav_count', 'qtd_tweeti

In [43]:
#Testing

In [ ]:
list(twitter_dataset['tweet_type'])

In [ ]:
for idx in twitter_dataset.index:
    print(twitter_dataset['tweet_type'][idx],twitter_dataset['rt_urls_list'][idx])

In [50]:
#Checking date format given in dataset

In [8]:
print('Date format:',twitter_dataset['date'][0].strip())
month_dict = {}
for idx in twitter_dataset.index:
    temp = twitter_dataset['date'][idx].split(' ')[1]
    if temp not in month_dict:
        month_dict[temp] = 1
print(month_dict)

Date format: Tue Feb 22 04:23:35 +0000 2022
{'Feb': 1, 'Mar': 1, 'Apr': 1}


In [53]:
#Input date from user

In [63]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [54]:
#Compiling urls tweet/retweet from dataset

In [64]:
tweetid_urldata = []
urllist_urldata = []
for idx in twitter_dataset.index:
    tweetid_urldata.append(twitter_dataset['tweetid'][idx])
    if twitter_dataset['tweet_type'][idx] == 'retweeted_tweet_without_comment':
        urllist_urldata.append(expanded_url_from(twitter_dataset['rt_urls_list'][idx]))
    else:
        urllist_urldata.append(expanded_url_from(twitter_dataset['urls_list'][idx]))

In [55]:
#Confirming length of url list and dataset size

In [31]:
len(urllist_urldata)

1302702

In [34]:
#Extracting attributes, url ids and classifying url types from twitter dataset on the requested date

In [142]:
tweetid_list = []
userid_list = []
screenname_list = []
date_list = []
url_list = []
url_id_list = []
url_type_list = []
count_ids = 0
count_cids = 0
count_nonids = 0
count = 0
#tweetid, userid, screen_name, date, url, url_id, url_type
for idx in twitter_dataset.index:
    date_from_dataset = twitter_dataset['date'][idx].split(' ')
    month_from_dataset = date_from_dataset[1]
    day_from_dataset = date_from_dataset[2]
    year_from_dataset = date_from_dataset[5]
    if month_from_dataset == month_query and day_from_dataset == day_query and year_from_dataset == year_query:
        tweet_id = twitter_dataset['tweetid'][idx]
        if tweet_id in tweetid_urldata:
            tweetid_index = tweetid_urldata.index(tweet_id)
            yt_url = urllist_urldata[tweetid_index]
            tweetid_list.append(tweet_id)
            userid_list.append(twitter_dataset['userid'][idx])
            screenname_list.append(twitter_dataset['screen_name'][idx])
            date_list.append(twitter_dataset['date'][idx])
            url_list.append(yt_url)
            link_type, link_id = getIDfromURL(yt_url)
            url_type_list.append(link_type)
            url_id_list.append(link_id)
            if link_type == 'other':
                count_nonids = count_nonids+1
            elif link_type == 'channel':
                count_cids = count_cids+1
            else:
                count_ids = count_ids+1
        count=count+1
        print(f'{count}/{len(twitter_dataset)} processed',end="\r")
dated_twitterset = pd.DataFrame({'tweetid': tweetid_list, 
                                 'userid': userid_list, 
                                 'screen_name': screenname_list, 
                                 'date': date_list,
                                 'url_id': url_id_list,
                                 'url':url_list, 
                                 'url_type':url_type_list })
dated_twitterset

476/1302702 processed

{'items': []}processed
{'items': []} processed


,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [56]:
#Verifying if all possible ids are extracted from the date queried dataset

In [125]:
count_cids,count_ids,count_nonids = countUrlType(dated_twitterset)
print(count_cids,count_ids,count_nonids)
checkSuccess(count_cids,count_ids,count_nonids, len(dated_twitterset))

118 17828 69


True

In [146]:
dated_twitterset.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv',index=False)

In [70]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [147]:
dated_twitterset = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv')

In [148]:
dated_twitterset

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [7]:
#Segregating different types of IDs for channel, videos and other

In [152]:
video_id_list = []
channel_id_list = []
other_id_list = []
for idx in dated_twitterset.index:
    if dated_twitterset['url_type'][idx] == 'channel':
        channel_id_list.append(dated_twitterset['url_id'][idx])
    elif dated_twitterset['url_type'][idx] == 'video':
        video_id_list.append(dated_twitterset['url_id'][idx])
    else:
        other_id_list.append(dated_twitterset['url_id'][idx])    

In [153]:
checkSuccess(len(channel_id_list),len(video_id_list),len(other_id_list), len(dated_twitterset))

True

In [108]:
#Gathering unique video ids from the list

In [156]:
videoid_set = set(video_id_list)
video_id_list = list(videoid_set)
channelid_set = set(channel_id_list)
channel_id_list = list(channelid_set)

In [110]:
#Generating comma separated list of video ids to curb YouTube Data API calls

In [161]:
nums_video = len(video_id_list)
vid_req_num = int(nums_video/50)
videoid_index = 0
cs_videoids = ''
vid_req_list = []
for ind in range(0,vid_req_num):
    for s_ind in range(videoid_index,videoid_index+50):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+50
if nums_video % 50 != 0:
    diff = nums_video - (50*vid_req_num)
    for s_ind in range(videoid_index,videoid_index+diff):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+diff
print(f'No. of unique videos {videoid_index}, no of API calls needed: {len(vid_req_list)}')    

No. of unique videos 6421, no of API calls needed: 129


In [112]:
#Generating comma separated list of channel ids to curb YouTube Data API calls

In [165]:
nums_channel = len(channel_id_list)
ch_req_num = int(nums_channel/50)
channelid_index = 0
cs_channelids = ''
cid_req_list = []
for ind in range(0,ch_req_num):
    for s_ind in range(channelid_index,channelid_index+50):
        if channel_id_list[s_ind] != 'NA' and str(channel_id_list[s_ind]) != 'nan':
            print(str(channel_id_list[s_ind]),end="\r")
            cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+50
if nums_channel % 50 != 0:
    diff = nums_channel - (50*ch_req_num)
    for s_ind in range(channelid_index,channelid_index+diff):
        if channel_id_list[s_ind] != 'NA' and str(channel_id_list[s_ind]) != 'nan':
            print(str(channel_id_list[s_ind]),end="\r")
            cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+diff
print(f'No of channels {channelid_index}, no of API calls needed: {len(cid_req_list)}')

No of channels 29, no of API calls needed: 1


In [81]:
#Making API calls for fetching video metadata

In [166]:
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[0]}&key={YT_API_KEY}')
json_data = json.loads(response_from.text)
if 'items' in json_data:
    for x in range(1,len(vid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[x]}&key={YT_API_KEY}')
        json_data_2 = json.loads(response2_from.text)
        if 'items' in json_data_2:
            json_data['items'] = [*json_data['items'],*json_data_2['items']]
        print(f'{x}/{len(vid_req_list)} processed',end="\r")
    print(len(json_data['items']))
else:
    print('some error occured fetching data from API')
    print(json_data)

5831129 processed


In [167]:
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics&id={cid_req_list[0]}&key={YT_API_KEY}')
channel_json_data = json.loads(response_from.text)
if 'items' in channel_json_data:
    for x in range(1,len(cid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={cid_req_list[x]}&key={YT_API_KEY}')
        channel_json_data_2 = json.loads(response2_from.text)
        if 'items' in channel_json_data_2:
            channel_json_data['items'] = [*channel_json_data['items'],*channel_json_data_2['items']]
        print(f'{x}/{len(cid_req_list)} processed',end="\r")
    print(len(channel_json_data['items']))
else:
    print('some error occured fetching data from API')
    print(channel_json_data)

28


In [84]:
#Extracting metadata from YouTube video JSON

In [168]:
v_url = []
v_channelId = []
v_channelTitle = []
v_tags=[]
v_views=[]
v_likes=[]
v_title=[]
v_lang = []
v_category = []
v_topic_categories = []
v_description=[]
v_publish_date=[]
v_comments=[]
v_videoTime = []
v_visited = []
v_contentRating = []
v_comments = []
unavailable_videoIds = []
response_id_list = []
data_list = json_data['items']
count = 0
for item in data_list:
    response_id_list.append(item['id'])
for id in video_id_list:
    v_url.append('https://www.youtube.com/watch?v='+id)
    if id not in response_id_list:
        v_channelTitle.append('NA')
        v_channelId.append('NA')
        v_tags.append(['NA'])
        v_lang.append('NA')
        v_category.append('NA')
        v_videoTime.append(0)
        v_views.append(0)
        v_likes.append(0)
        v_topic_categories.append(['NA'])
        v_comments.append(0)
        v_title.append('NA')
        v_description.append('NA')
        v_publish_date.append('NA')
        v_contentRating.append('NA')
    else:
        v_index = response_id_list.index(id)
        if 'channelTitle' in json_data['items'][v_index]['snippet']:
            v_channelTitle.append(json_data['items'][v_index]['snippet']['channelTitle'])
        else:
            v_channelTitle.append('NA')
        if 'channelId' in json_data['items'][v_index]['snippet']:
            v_channelId.append(json_data['items'][v_index]['snippet']['channelId'])
        else:
            v_channelId.append('NA')
        if 'tags' in json_data['items'][v_index]['snippet']:
            v_tags.append(json_data['items'][v_index]['snippet']['tags'])
        else:
            v_tags.append(['NA'])
        if 'defaultLanguage' in json_data['items'][v_index]['snippet']:
            v_lang.append(json_data['items'][v_index]['snippet']['defaultLanguage'][0:2])
        else:
            v_lang.append('NA')
        if 'categoryId' in json_data['items'][v_index]['snippet']:
            v_category.append(get_video_category_from_id(json_data['items'][v_index]['snippet']['categoryId']))
        else:
            v_category.append('NA')
        if 'duration' in json_data['items'][v_index]['contentDetails']:
                #print(item['id'])
            v_videoTime.append(convert_time_to_sec(json_data['items'][v_index]['contentDetails']['duration']))
        else:
            v_videoTime.append(0)
        if 'viewCount' in json_data['items'][v_index]['statistics']:
            v_views.append(int(json_data['items'][v_index]['statistics']['viewCount']))
        else:
            v_views.append(0)
        if 'likeCount' in json_data['items'][v_index]['statistics']:
            v_likes.append(int(json_data['items'][v_index]['statistics']['likeCount']))
        else:
            v_likes.append(0)
        if 'topicDetails' in json_data['items'][v_index] and 'topicCategories' in json_data['items'][v_index]['topicDetails']:
            cleaned_topic_list = list(map(lambda i:get_topic_category_from_wiki(json_data['items'][v_index]['topicDetails']['topicCategories'][i]), range(0, len(json_data['items'][v_index]['topicDetails']['topicCategories']))))
            v_topic_categories.append(cleaned_topic_list)
        else:
            v_topic_categories.append(['NA'])
        if 'commentCount' in json_data['items'][v_index]['statistics']:
            v_comments.append(int(json_data['items'][v_index]['statistics']['commentCount']))
        else:
            v_comments.append(0)
        if 'title' in json_data['items'][v_index]['snippet']:
            v_title.append(json_data['items'][v_index]['snippet']['title'])
        else:
            v_title.append('NA')
        if 'description' in json_data['items'][v_index]['snippet'] and len(json_data['items'][v_index]['snippet']['description']) != 0:
            v_description.append(json_data['items'][v_index]['snippet']['description'])
        else:
            v_description.append('NA')
        if 'publishedAt' in json_data['items'][v_index]['snippet']:
            v_publish_date.append(json_data['items'][v_index]['snippet']['publishedAt'])
        else:
            v_publish_date.append('NA')
        if 'contentDetails' in json_data['items'][v_index] and 'contentRating' in json_data['items'][v_index]['contentDetails']:
            v_contentRating.append(json_data['items'][v_index]['contentDetails']['contentRating'])
        else:
            v_contentRating.append('NA')
        #v_comments.append(getCommentsForVideo(id))
    count = count+1
    print(f'{count}/{videoid_index} processed',end="\r")
yt_data = pd.DataFrame({'videoId': video_id_list,
                        'videoURL': v_url,
                        'videoTitle':v_title,
                        'videoDescription':v_description,
                        'videoPublishedAt':v_publish_date,
                        'channelId':v_channelId,
                         'channelTitle':v_channelTitle,
                         'videoTime':v_videoTime,
                        'videoTags':v_tags,
                        'videoCategory':v_category,
                        'videoTopics':v_topic_categories,
                        'videoLanguage':v_lang,
                        'videoViews':v_views,
                        'videoLikes':v_likes,
                        'videoComments':v_comments,
                        'videoContentRating':v_contentRating,
                       })
yt_data

,videoId,videoURL,videoTitle,videoDescription,videoPublishedAt,channelId,channelTitle,videoTime,videoTags,videoCategory,videoTopics,videoLanguage,videoViews,videoLikes,videoComments,videoContentRating
0,eOqZ4ejpSYI,https://www.youtube.com/watch?v=eOqZ4ejpSYI,russian army 2020 الجيش الروسي русская армия,Russian army 2020\nالجيش الروسي 2020,2019-06-26T13:26:03Z,UCfUu8yOs6IXBrROcsl9R30g,السيء,581,[NA],People & Blogs,"[Military, Society]",NA,3797859,41782,5686,{}
1,aA1npJdN3M4,https://www.youtube.com/watch?v=aA1npJdN3M4,Psaki probed over if sanctions with Russia wil...,White House Press Secretary Jen Psaki has been...,2022-02-23T05:00:50Z,UCO0akufu9MOzyz3nvGIXAAw,Sky News Australia,166,"[6298555357001, fb, global, msn, world, yt]",News & Politics,"[Politics, Society]",NA,8216,195,205,{}
2,5HHrkcXXfWk,https://www.youtube.com/watch?v=5HHrkcXXfWk,ウクライナ人女性が話す【１８歳から日本人男性が好きになった理由４選】,過去にさかのぼって、なぜ日本人の男性に興味を持ったかお話しします。\n全てのお話しは個人意見...,2022-02-22T10:30:01Z,UCdL7FtmjBkgPuroig2UiTJA,クリススタイルチャンネル,870,[NA],People & Blogs,[Lifestyle_(sociology)],NA,94800,3092,108,{}
3,SnoEctYVOWU,https://www.youtube.com/watch?v=SnoEctYVOWU,Putin will stop at nothing to bring back the S...,Eric Bolling exposes Russian President Vladimi...,2022-02-22T22:00:23Z,UCx6h-dWzJ5NpAlja1YsApdg,Newsmax TV,443,"[Newsmax, News, Politics, Conservative, Washin...",News & Politics,"[Politics, Society]",NA,7034,370,232,{}
4,SKSvPaT6V9k,https://www.youtube.com/watch?v=SKSvPaT6V9k,Aida Garifullina & Dmitri Hvorostovsky - Deja ...,Aida Garifullina and Dmitri Hvorostovsky\nNew ...,2015-10-12T09:10:06Z,UCNt7TnGQT0n3dNT83MsR1lA,Aida Garifullina,387,"[Aida Garifullina, Dmitri Hvorostovsky, new wa...",Music,"[Classical_music, Music, Pop_music]",NA,2578889,23869,1391,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6416,5y_ktAuCypQ,https://www.youtube.com/watch?v=5y_ktAuCypQ,5 reasons Putin's Russia won't have it easy ag...,Play War Thunder for FREE! Register using http...,2022-01-30T22:00:02Z,UCKNCbBWiMiXBVXUmUuu_dsQ,Defense Updates,505,"[ukraine, nato, putin, russia, russia ukraine,...",Science & Technology,"[Military, Society]",en,110708,2931,739,{}
6417,opPwDz04kzI,https://www.youtube.com/watch?v=opPwDz04kzI,FULL VIDEO: Trump and Putin Speak After Summit,President Trump and President Vladimir V. Puti...,2018-07-16T16:07:11Z,UCqnbDFdCpuN8CMEg0VuEBqA,The New York Times,5364,"[trump and russia, trump and russia speak, tru...",News & Politics,"[Politics, Society]",NA,73087,550,333,{}
6418,lA3lGR-_sc8,https://www.youtube.com/watch?v=lA3lGR-_sc8,Russia Moves to Recognise Donbass Republics as...,Russia Moves to Recognise Donbass Republics as...,2022-02-21T21:09:58Z,UCwGpHa6rMLjSSCBlckm5khw,Alexander Mercouris,2583,"[Alexander Mercouris, The Duran, duran news, a...",News & Politics,"[Religion, Society]",NA,46494,3552,790,{}
6419,7lrfdzU8k4k,https://www.youtube.com/watch?v=7lrfdzU8k4k,Deadly AC-130 Gunship in Action Firing All Its...,The AC-130 is the world's biggest flying artil...,2018-08-03T15:30:01Z,UCWqPRUsJlZaDp-PVbqEch9g,Smithsonian Channel,188,"[Hell in the sky, TV online, watch, center ope...",Entertainment,[Society],NA,4906723,68067,4445,{}


In [86]:
#Extracting metadata for YouTube channels from JSON

In [172]:
vc_channelTitle = []
vc_topic_categories = []
vc_subs=[]
vc_views=[]
vc_vidcnt=[]
vc_cntry=[]
vc_description=[]
vc_publish_date=[]
vc_visited = []
vc_url = []
data_list = channel_json_data['items']
response_cid_list = []
count = 0
for item in data_list:
    response_cid_list.append(item['id'])
for id in channel_id_list:
    if id != 'NA' and str(id) != 'nan':
        vc_url.append('https://www.youtube.com/c/'+id)
        if id not in response_cid_list:
            vc_channelTitle.append('NA')
            vc_topic_categories.append(['NA'])
            vc_subs.append('NA')
            vc_views.append('NA')
            vc_vidcnt.append('NA')
            vc_cntry.append('NA')
            vc_description.append('NA')
            vc_publish_date.append('NA')
        else:
            v_index = response_cid_list.index(id)
            if 'title' in channel_json_data['items'][v_index]['snippet'] and :
                vc_channelTitle.append(channel_json_data['items'][v_index]['snippet']['title'])
            else:
                vc_channelTitle.append('NA')
            if 'topicDetails' in channel_json_data['items'][v_index] and 'topicCategories' in channel_json_data['items'][v_index]['topicDetails']:
                cleaned_topic_list = list(
                    map(
                        lambda i:get_topic_category_from_wiki(
                            channel_json_data['items'][v_index]['topicDetails']['topicCategories'][i]), 
                        range(0, len(channel_json_data['items'][v_index]['topicDetails']['topicCategories']))))
                vc_topic_categories.append(cleaned_topic_list)
            else:
                vc_topic_categories.append(['NA'])
            if 'subscriberCount' in channel_json_data['items'][v_index]['statistics']:
                vc_subs.append(int(channel_json_data['items'][v_index]['statistics']['subscriberCount']))
            else:
                vc_subs.append(0)
            if 'viewCount' in channel_json_data['items'][v_index]['statistics']:
                vc_views.append(int(channel_json_data['items'][v_index]['statistics']['viewCount']))
            else:
                vc_views.append(0)
            if 'videoCount' in channel_json_data['items'][v_index]['statistics']:
                vc_vidcnt.append(int(channel_json_data['items'][v_index]['statistics']['videoCount']))
            else:
                vc_vidcnt.append(0)
            if 'country' in channel_json_data['items'][v_index]['snippet']:
                vc_cntry.append(channel_json_data['items'][v_index]['snippet']['country'])
            else:
                vc_cntry.append('NA')
            if 'description' in channel_json_data['items'][v_index]['snippet']:
                vc_description.append(channel_json_data['items'][v_index]['snippet']['description'])
            else:
                vc_description.append('NA')
            if 'publishedAt' in channel_json_data['items'][v_index]['snippet']:
                vc_publish_date.append(channel_json_data['items'][v_index]['snippet']['publishedAt'])
            else:
                vc_publish_date.append('NA')
    count = count+1
    print(f'{count}/{videoid_index} processed',end="\r")
yt_channel_data = pd.DataFrame({'channelTitle':vc_channelTitle,
                                         'channelDescription':vc_description,
                                         'subscribers':vc_subs,
                                         'url':vc_url,
                                        'subscribers':vc_subs,
                                        'channelViews':vc_views,
                                        'videoCount':vc_vidcnt,
                                        'channelCountry':vc_cntry,
                                        'channelTopicCategory':vc_topic_categories,
                                        'channelPublishedAt':vc_publish_date})
yt_channel_data

,channelTitle,channelDescription,subscribers,url,channelViews,videoCount,channelCountry,channelTopicCategory,channelPublishedAt
0,FUTURE CONFLICT,Future Conflict is a YouTube channel focusing ...,2810,https://www.youtube.com/c/UCD6f-qHBtzBrxnMAeGC...,107929,148,US,[NA],2006-08-21T02:00:24Z
1,The English Channel Of Al-Mahdi Nasser Mohammed,This channel tackles the translated into Engli...,1850,https://www.youtube.com/c/UCbzTU36Nmiz8UXpdG3R...,70783,101,EG,[NA],2021-09-10T11:26:27.823875Z
2,Eichhorn Coaching,Alexander Eichhorn und Maximilian Bothe sind e...,5770,https://www.youtube.com/c/UCzEnX97Kzb9G-Zrpl7E...,743072,453,DE,[NA],2018-02-08T17:18:13Z
3,Gabriele Krone-Schmalz,,3790,https://www.youtube.com/c/UC9bIzo1UsNGjAIiwtAc...,125056,15,NA,[NA],2015-09-22T13:35:39Z
4,AZOV media,Офіційний канал полку «Азов»\n\nПолк «Азов» од...,159000,https://www.youtube.com/c/UCewl92lzIMDO8QiAYOQ...,35107430,1030,UA,[NA],2014-08-05T17:47:45Z
5,Церковь Еммануил,"Христианские проповеди, прославление. Церковь ...",18400,https://www.youtube.com/c/UCu0e3kdu2SrV6bjolfC...,4809713,2783,UA,[NA],2014-05-23T13:20:13Z
6,Juan1890,,64,https://www.youtube.com/c/UCEvsaEJWYMcjdVJtYNK...,1489,27,NA,[NA],2019-09-04T20:18:46Z
7,bald and bankrupt,First Englishman to be made a Thief in Law by ...,3630000,https://www.youtube.com/c/UCxDZs_ltFFvn0FDHT6k...,527564584,240,GB,[NA],2018-06-12T11:56:25Z
8,TommaHawk79 Games,"Aussie Gamer i Livestream Fifa 20 , 21 & F1 2...",527,https://www.youtube.com/c/UCqO_7yPSaFdHkjf5Bt2...,79819,2177,AU,[NA],2017-02-07T11:43:33Z
9,Liv Romano,,112,https://www.youtube.com/c/UCZN6grU_dkwYshYQwPi...,1042,21,NA,[NA],2021-09-18T01:44:41.25914Z


In [178]:
#Saving channel and video metadata as per their date

In [174]:
yt_channel_data.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}ChannelDataset.csv',index=False)

In [ ]:
yt_data.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}VideoDataset.csv',index=False)

In [15]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [176]:
yt_channel_data = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}ChannelDataset.csv')
yt_channel_data

,channelTitle,channelDescription,subscribers,url,channelViews,videoCount,channelCountry,channelTopicCategory,channelPublishedAt
0,FUTURE CONFLICT,Future Conflict is a YouTube channel focusing ...,2810,https://www.youtube.com/c/UCD6f-qHBtzBrxnMAeGC...,107929,148,US,['NA'],2006-08-21T02:00:24Z
1,The English Channel Of Al-Mahdi Nasser Mohammed,This channel tackles the translated into Engli...,1850,https://www.youtube.com/c/UCbzTU36Nmiz8UXpdG3R...,70783,101,EG,['NA'],2021-09-10T11:26:27.823875Z
2,Eichhorn Coaching,Alexander Eichhorn und Maximilian Bothe sind e...,5770,https://www.youtube.com/c/UCzEnX97Kzb9G-Zrpl7E...,743072,453,DE,['NA'],2018-02-08T17:18:13Z
3,Gabriele Krone-Schmalz,NaN,3790,https://www.youtube.com/c/UC9bIzo1UsNGjAIiwtAc...,125056,15,NaN,['NA'],2015-09-22T13:35:39Z
4,AZOV media,Офіційний канал полку «Азов»\n\nПолк «Азов» од...,159000,https://www.youtube.com/c/UCewl92lzIMDO8QiAYOQ...,35107430,1030,UA,['NA'],2014-08-05T17:47:45Z
5,Церковь Еммануил,"Христианские проповеди, прославление. Церковь ...",18400,https://www.youtube.com/c/UCu0e3kdu2SrV6bjolfC...,4809713,2783,UA,['NA'],2014-05-23T13:20:13Z
6,Juan1890,NaN,64,https://www.youtube.com/c/UCEvsaEJWYMcjdVJtYNK...,1489,27,NaN,['NA'],2019-09-04T20:18:46Z
7,bald and bankrupt,First Englishman to be made a Thief in Law by ...,3630000,https://www.youtube.com/c/UCxDZs_ltFFvn0FDHT6k...,527564584,240,GB,['NA'],2018-06-12T11:56:25Z
8,TommaHawk79 Games,"Aussie Gamer i Livestream Fifa 20 , 21 & F1 2...",527,https://www.youtube.com/c/UCqO_7yPSaFdHkjf5Bt2...,79819,2177,AU,['NA'],2017-02-07T11:43:33Z
9,Liv Romano,NaN,112,https://www.youtube.com/c/UCZN6grU_dkwYshYQwPi...,1042,21,NaN,['NA'],2021-09-18T01:44:41.25914Z


In [16]:
yt_data = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}VideoDataset.csv')
yt_data

,videoId,videoURL,videoTitle,videoDescription,videoPublishedAt,channelId,channelTitle,videoTime,videoTags,videoCategory,videoTopics,videoLanguage,videoViews,videoLikes,videoComments,videoContentRating
0,eOqZ4ejpSYI,https://www.youtube.com/watch?v=eOqZ4ejpSYI,russian army 2020 الجيش الروسي русская армия,Russian army 2020\nالجيش الروسي 2020,2019-06-26T13:26:03Z,UCfUu8yOs6IXBrROcsl9R30g,السيء,581,['NA'],People & Blogs,"['Military', 'Society']",NaN,3797859,41782,5686,{}
1,aA1npJdN3M4,https://www.youtube.com/watch?v=aA1npJdN3M4,Psaki probed over if sanctions with Russia wil...,White House Press Secretary Jen Psaki has been...,2022-02-23T05:00:50Z,UCO0akufu9MOzyz3nvGIXAAw,Sky News Australia,166,"['6298555357001', 'fb', 'global', 'msn', 'worl...",News & Politics,"['Politics', 'Society']",NaN,8216,195,205,{}
2,5HHrkcXXfWk,https://www.youtube.com/watch?v=5HHrkcXXfWk,ウクライナ人女性が話す【１８歳から日本人男性が好きになった理由４選】,過去にさかのぼって、なぜ日本人の男性に興味を持ったかお話しします。\n全てのお話しは個人意見...,2022-02-22T10:30:01Z,UCdL7FtmjBkgPuroig2UiTJA,クリススタイルチャンネル,870,['NA'],People & Blogs,['Lifestyle_(sociology)'],NaN,94800,3092,108,{}
3,SnoEctYVOWU,https://www.youtube.com/watch?v=SnoEctYVOWU,Putin will stop at nothing to bring back the S...,Eric Bolling exposes Russian President Vladimi...,2022-02-22T22:00:23Z,UCx6h-dWzJ5NpAlja1YsApdg,Newsmax TV,443,"['Newsmax', 'News', 'Politics', 'Conservative'...",News & Politics,"['Politics', 'Society']",NaN,7034,370,232,{}
4,SKSvPaT6V9k,https://www.youtube.com/watch?v=SKSvPaT6V9k,Aida Garifullina & Dmitri Hvorostovsky - Deja ...,Aida Garifullina and Dmitri Hvorostovsky\nNew ...,2015-10-12T09:10:06Z,UCNt7TnGQT0n3dNT83MsR1lA,Aida Garifullina,387,"['Aida Garifullina', 'Dmitri Hvorostovsky', 'n...",Music,"['Classical_music', 'Music', 'Pop_music']",NaN,2578889,23869,1391,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6416,5y_ktAuCypQ,https://www.youtube.com/watch?v=5y_ktAuCypQ,5 reasons Putin's Russia won't have it easy ag...,Play War Thunder for FREE! Register using http...,2022-01-30T22:00:02Z,UCKNCbBWiMiXBVXUmUuu_dsQ,Defense Updates,505,"['ukraine', 'nato', 'putin', 'russia', 'russia...",Science & Technology,"['Military', 'Society']",en,110708,2931,739,{}
6417,opPwDz04kzI,https://www.youtube.com/watch?v=opPwDz04kzI,FULL VIDEO: Trump and Putin Speak After Summit,President Trump and President Vladimir V. Puti...,2018-07-16T16:07:11Z,UCqnbDFdCpuN8CMEg0VuEBqA,The New York Times,5364,"['trump and russia', 'trump and russia speak',...",News & Politics,"['Politics', 'Society']",NaN,73087,550,333,{}
6418,lA3lGR-_sc8,https://www.youtube.com/watch?v=lA3lGR-_sc8,Russia Moves to Recognise Donbass Republics as...,Russia Moves to Recognise Donbass Republics as...,2022-02-21T21:09:58Z,UCwGpHa6rMLjSSCBlckm5khw,Alexander Mercouris,2583,"['Alexander Mercouris', 'The Duran', 'duran ne...",News & Politics,"['Religion', 'Society']",NaN,46494,3552,790,{}
6419,7lrfdzU8k4k,https://www.youtube.com/watch?v=7lrfdzU8k4k,Deadly AC-130 Gunship in Action Firing All Its...,The AC-130 is the world's biggest flying artil...,2018-08-03T15:30:01Z,UCWqPRUsJlZaDp-PVbqEch9g,Smithsonian Channel,188,"['Hell in the sky', 'TV online', 'watch', 'cen...",Entertainment,['Society'],NaN,4906723,68067,4445,{}


In [20]:
videoid_list = list(yt_data['videoId'])
comment_list = []
videoid_len = len(videoid_list)
print(f'Number of videos: {videoid_len}')
now = datetime.now()
dt_string = now.strftime("%H:%M:%S")
print(f'Started at: {dt_string}')
for idx,vid in enumerate(videoid_list):
    comment_list.append(getCommentsForVideo(vid))
    print(f'{idx+1}/{videoid_len} processed',end="\r")
dt_string = now.strftime("%H:%M:%S")
print(f'Ended at: {dt_string}')
print(comment_list)

Number of videos: 6421
Started at: 18:40:09
Ended at: 18:40:09d


In [21]:
print(comment_list)